# Pure Python ReAct Agent with Intelligent Search Trigger

This notebook demonstrates a single-agent AI system that uses the ReAct (Reason + Act) pattern.
The agent can intelligently decide when to search the web vs when to use its built-in knowledge.

**Key Features:**
- Intelligent search trigger based on temporal keywords and topics
- Integration with Gemini Flash 2.0 for reasoning
- Web search capability via Serper API
- ReAct pattern implementation (Reason → Act → Observe)

**Workshop Demo - Single Agent AI System**

## Install Dependencies

First, let's install all the required packages for our ReAct agent.

In [5]:
# Install required packages
!pip install google-generativeai requests tavily-python -q

## Setup API Keys with Serper Dev

Configure your API keys for Gemini and Serper. In Colab, use the secrets panel to store your keys securely.

In [24]:
import os
from google.colab import userdata

# Set up API keys from Colab secrets
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

print("API keys configured successfully!")

API keys configured successfully!


## Setup API Keys with Tavily API Key

In [29]:
import os
from google.colab import userdata

# Set up API keys from Colab secrets
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

print("API keys configured successfully!")

API keys configured successfully!


## Search Trigger Intelligence Module

This is the core decision-making component that automatically identifies when a web search is necessary using pattern recognition.

In [30]:
import re
from typing import Dict, List, Any, Optional

class SearchTriggerIntelligence:
    """
    The Search Trigger Intelligence module is the system's core decision-making component.
    It automatically identifies when a web search is necessary using pattern recognition
    rather than simple keyword matching.
    """

    def __init__(self):
        # Time-sensitive keywords that indicate need for current information
        self.temporal_keywords = {
            'immediate': ['now', 'currently', 'today', 'this week', 'right now'],
            'recent': ['latest', 'recent', 'new', 'fresh', 'updated', 'current'],
            'trending': ['trending', 'popular', 'viral', 'breaking', 'hot'],
            'temporal_markers': ['2025', '2024', 'this year', 'next year'],
            'news_indicators': ['news', 'developments', 'updates', 'announcement']
        }

        # Topics that require real-time information updates
        self.current_info_topics = {
            'technology': ['ai', 'artificial intelligence', 'tech', 'software', 'app', 'startup'],
            'finance': ['market', 'stock', 'crypto', 'bitcoin', 'economy', 'price'],
            'news': ['politics', 'election', 'government', 'policy', 'war'],
            'science': ['research', 'study', 'discovery', 'breakthrough', 'covid'],
            'events': ['conference', 'pycon', 'summit', 'meeting', 'event', 'happening'],
            'weather': ['weather', 'temperature', 'forecast', 'climate']
        }

        # Question patterns that typically need search
        self.search_patterns = [
            r'where is .* happening',
            r'when is .* scheduled',
            r'what is the latest',
            r'tell me about recent',
            r'current status of',
            r'how much does .* cost',
            r'what are the reviews'
        ]

    def needs_search(self, query: str) -> bool:
        """
        Analyze if a query needs web search based on multiple factors.

        Args:
            query (str): User's input query

        Returns:
            bool: True if search is needed, False otherwise
        """
        query_lower = query.lower()

        # Check for temporal keywords
        for category, keywords in self.temporal_keywords.items():
            if any(keyword in query_lower for keyword in keywords):
                print(f"Search triggered by temporal keyword ({category})")
                return True

        # Check for current info topics
        for category, topics in self.current_info_topics.items():
            if any(topic in query_lower for topic in topics):
                print(f"Search triggered by topic category ({category})")
                return True

        # Check for search patterns using regex
        for pattern in self.search_patterns:
            if re.search(pattern, query_lower):
                print(f"Search triggered by pattern: {pattern}")
                return True

        # If no triggers found, use built-in knowledge
        print("Using built-in knowledge (no search needed)")
        return False

print("Search Trigger Intelligence module loaded!")

Search Trigger Intelligence module loaded!


## Web Search Tool via Serper Dev

Web search functionality using Serper API for getting real-time information from Google.

In [ ]:
import json
import requests

class WebSearchTool:
    """
    Web search functionality using Serper API.
    Provides structured search results from Google.
    """

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://google.serper.dev/search"

    def search(self, query: str, num_results: int = 5) -> Dict[str, Any]:
        """
        Perform web search using Serper API.

        Args:
            query (str): Search query
            num_results (int): Number of results to return

        Returns:
            Dict containing search results
        """
        try:
            payload = json.dumps({"q": query, "num": num_results})
            headers = {
                'X-API-KEY': self.api_key,
                'Content-Type': 'application/json'
            }

            response = requests.post(self.base_url, headers=headers, data=payload)
            response.raise_for_status()

            return response.json()

        except requests.exceptions.RequestException as e:
            return {"error": f"Search failed: {str(e)}"}

    def format_results(self, search_results: Dict[str, Any]) -> str:
        """
        Format search results into readable text.

        Args:
            search_results (Dict): Raw search results from API

        Returns:
            str: Formatted search results
        """
        if "error" in search_results:
            return f"Search Error: {search_results['error']}"

        if "organic" not in search_results:
            return "No search results found."

        formatted = "🔍 Web Search Results:\n\n"

        for i, result in enumerate(search_results["organic"][:5], 1):
            title = result.get("title", "No title")
            snippet = result.get("snippet", "No description")
            link = result.get("link", "No link")

            formatted += f"{i}. **{title}**\n"
            formatted += f"   {snippet}\n"
            formatted += f"   Source: {link}\n\n"

        return formatted

print("Web Search Tool loaded!")

Web Search Tool loaded!


## Web Search tool using Tavily API Key

In [31]:
import json
from typing import Any, Dict
from tavily import TavilyClient

class WebSearchTool:
    """
    Web search functionality using Tavily API.
    Provides structured search results similar to the original Serper tool.
    """

    def __init__(self, api_key: str):
        self.api_key = api_key
        # Tavily client wraps the API endpoint for us
        self.client = TavilyClient(self.api_key)

    def search(self, query: str, num_results: int = 5) -> Dict[str, Any]:
        """
        Perform web search using Tavily API.

        Args:
            query (str): Search query
            num_results (int): Number of results to return (sliced client-side)

        Returns:
            Dict containing search results (or {"error": "..."} on failure)
        """
        try:
            # TavilyClient.search(...) returns a dict-like object (see your sample)
            # Pass query; if TavilyClient supports extra params you can add them here.
            response = self.client.search(query=query)

            # Ensure response is a dict
            if not isinstance(response, dict):
                return {"error": "Unexpected response format from Tavily API", "raw": response}

            # Normalize / slice results to num_results to keep behaviour consistent
            if "results" in response and isinstance(response["results"], list):
                response["results"] = response["results"][:num_results]

            return response

        except Exception as e:
            return {"error": f"Search failed: {str(e)}"}

    def format_results(self, search_results: Dict[str, Any]) -> str:
        """
        Format search results into readable text.

        Args:
            search_results (Dict): Raw search results from API

        Returns:
            str: Formatted search results
        """
        if "error" in search_results:
            return f"Search Error: {search_results['error']}"

        # Tavily returns results under "results" (based on your example)
        if "results" not in search_results:
            return "No search results found."

        formatted = "🔍 Web Search Results:\n\n"

        for i, result in enumerate(search_results["results"][:5], 1):
            # Map Tavily fields to the same names you used previously
            title = result.get("title", "No title")
            # Tavily uses "content" (or raw_content) as snippet-like text
            snippet = result.get("content") or result.get("raw_content") or "No description"
            link = result.get("url") or result.get("link") or "No link"

            formatted += f"{i}. **{title}**\n"
            formatted += f"   {snippet}\n"
            formatted += f"   Source: {link}\n\n"

        return formatted

print("Web Search Tool loaded!")

Web Search Tool loaded!


## Gemini LLM Integration

Gemini Flash 2.0 integration for reasoning and response generation - the 'thinking' part of the ReAct pattern.

In [32]:
import google.generativeai as genai

class GeminiLLM:
    """
    Gemini Flash 2.0 integration for reasoning and response generation.
    Handles the 'thinking' part of the ReAct pattern.
    """

    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash-exp')

    def generate_response(self, prompt: str) -> str:
        """
        Generate response using Gemini model.

        Args:
            prompt (str): Input prompt for the model

        Returns:
            str: Generated response
        """
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating response: {str(e)}"

print("Gemini LLM integration loaded!")

Gemini LLM integration loaded!


## Pure ReAct Agent

Main ReAct Agent that combines reasoning and action capabilities using the ReAct pattern:
1. **REASON**: Analyze the query and decide what action to take
2. **ACT**: Execute the action (search web or use knowledge)
3. **OBSERVE**: Process the results and generate final response

In [36]:
import os
from typing import Any, Dict

class PureReActAgentTavily:
    """
    Main ReAct Agent that combines reasoning and action capabilities.

    The ReAct pattern works as follows:
    1. REASON: Analyze the query and decide what action to take
    2. ACT: Execute the action (search web or use knowledge)
    3. OBSERVE: Process the results and generate final response
    """

    def __init__(self):
        # Initialize components with API keys from environment
        gemini_key = os.getenv("GEMINI_API_KEY")
        tavily_key = os.getenv("TAVILY_API_KEY")

        if not gemini_key or not tavily_key:
            raise ValueError("Missing API keys in environment variables")

        self.llm = GeminiLLM(gemini_key)
        self.search_tool = WebSearchTool(tavily_key)
        self.search_trigger = SearchTriggerIntelligence()

        print("ReAct Agent initialized successfully!")
        print("Ready to reason and act on your queries!")

    def reason(self, query: str) -> Dict[str, Any]:
        """
        REASON step: Analyze the query and decide on action.

        Args:
            query (str): User's input query

        Returns:
            Dict containing reasoning results
        """
        print(f"\n REASONING about: '{query}'")

        # Use search trigger intelligence to decide
        needs_search = self.search_trigger.needs_search(query)

        reasoning = {
            "query": query,
            "needs_search": needs_search,
            "action": "web_search" if needs_search else "direct_response",
            "reasoning": f"Query {'requires' if needs_search else 'does not require'} web search"
        }

        print(f"💭 Decision: {reasoning['action']}")
        return reasoning

    def act(self, reasoning: Dict[str, Any]) -> Dict[str, Any]:
        """
        ACT step: Execute the decided action.

        Args:
            reasoning (Dict): Results from reasoning step

        Returns:
            Dict containing action results
        """
        query = reasoning["query"]
        action = reasoning["action"]

        print(f"\n⚡ ACTING: {action}")

        if action == "web_search":
            # Perform web search
            search_results = self.search_tool.search(query)
            formatted_results = self.search_tool.format_results(search_results)

            return {
                "action": action,
                "results": formatted_results,
                "raw_data": search_results
            }

        else:
            # Use direct LLM response
            prompt = f"""
            Answer the following question using your built-in knowledge:

            Question: {query}

            Provide a helpful and informative response.
            """

            response = self.llm.generate_response(prompt)

            return {
                "action": action,
                "results": response,
                "raw_data": None
            }

    def observe_and_respond(self, query: str, action_results: Dict[str, Any]) -> str:
        """
        OBSERVE step: Process action results and generate final response.

        Args:
            query (str): Original user query
            action_results (Dict): Results from action step

        Returns:
            str: Final response to user
        """
        print(f"\n OBSERVING results and generating response...")

        if action_results["action"] == "web_search":
            # Synthesize search results into coherent response
            prompt = f"""
            Based on the following web search results, provide a comprehensive answer to the user's question.

            User Question: {query}

            Search Results:
            {action_results["results"]}

            Instructions:
            - Synthesize the information from multiple sources
            - Provide specific details and facts
            - Mention sources when relevant
            - Keep the response informative but concise
            """

            final_response = self.llm.generate_response(prompt)

        else:
            # Direct response from LLM
            final_response = action_results["results"]

        return final_response

    def process_query(self, query: str) -> str:
        """
        Main method that implements the complete ReAct cycle.

        Args:
            query (str): User's input query

        Returns:
            str: Final response
        """
        print("="*60)
        print(" Starting ReAct Process")
        print("="*60)

        try:
            # Step 1: REASON
            reasoning = self.reason(query)

            # Step 2: ACT
            action_results = self.act(reasoning)

            # Step 3: OBSERVE and respond
            final_response = self.observe_and_respond(query, action_results)

            print(f"\n FINAL RESPONSE:")
            print("-" * 40)
            return final_response

        except Exception as e:
            error_msg = f" Error in ReAct process: {str(e)}"
            print(error_msg)
            return error_msg

print(" Pure ReAct Agent class loaded with Tavily!")

 Pure ReAct Agent class loaded with Tavily!


In [25]:
class PureReActAgentSerper:
    """
    Main ReAct Agent that combines reasoning and action capabilities.

    The ReAct pattern works as follows:
    1. REASON: Analyze the query and decide what action to take
    2. ACT: Execute the action (search web or use knowledge)
    3. OBSERVE: Process the results and generate final response
    """

    def __init__(self):
        # Initialize components with API keys from environment
        gemini_key = os.getenv("GEMINI_API_KEY")
        serper_key = os.getenv("SERPER_API_KEY")

        if not gemini_key or not serper_key:
            raise ValueError("Missing API keys in environment variables")

        self.llm = GeminiLLM(gemini_key)
        self.search_tool = WebSearchTool(serper_key)
        self.search_trigger = SearchTriggerIntelligence()

        print("ReAct Agent initialized successfully!")
        print("Ready to reason and act on your queries!")

    def reason(self, query: str) -> Dict[str, Any]:
        """
        REASON step: Analyze the query and decide on action.

        Args:
            query (str): User's input query

        Returns:
            Dict containing reasoning results
        """
        print(f"\n REASONING about: '{query}'")

        # Use search trigger intelligence to decide
        needs_search = self.search_trigger.needs_search(query)

        reasoning = {
            "query": query,
            "needs_search": needs_search,
            "action": "web_search" if needs_search else "direct_response",
            "reasoning": f"Query {'requires' if needs_search else 'does not require'} web search"
        }

        print(f"💭 Decision: {reasoning['action']}")
        return reasoning

    def act(self, reasoning: Dict[str, Any]) -> Dict[str, Any]:
        """
        ACT step: Execute the decided action.

        Args:
            reasoning (Dict): Results from reasoning step

        Returns:
            Dict containing action results
        """
        query = reasoning["query"]
        action = reasoning["action"]

        print(f"\n⚡ ACTING: {action}")

        if action == "web_search":
            # Perform web search
            search_results = self.search_tool.search(query)
            formatted_results = self.search_tool.format_results(search_results)

            return {
                "action": action,
                "results": formatted_results,
                "raw_data": search_results
            }

        else:
            # Use direct LLM response
            prompt = f"""
            Answer the following question using your built-in knowledge:

            Question: {query}

            Provide a helpful and informative response.
            """

            response = self.llm.generate_response(prompt)

            return {
                "action": action,
                "results": response,
                "raw_data": None
            }

    def observe_and_respond(self, query: str, action_results: Dict[str, Any]) -> str:
        """
        OBSERVE step: Process action results and generate final response.

        Args:
            query (str): Original user query
            action_results (Dict): Results from action step

        Returns:
            str: Final response to user
        """
        print(f"\n OBSERVING results and generating response...")

        if action_results["action"] == "web_search":
            # Synthesize search results into coherent response
            prompt = f"""
            Based on the following web search results, provide a comprehensive answer to the user's question.

            User Question: {query}

            Search Results:
            {action_results["results"]}

            Instructions:
            - Synthesize the information from multiple sources
            - Provide specific details and facts
            - Mention sources when relevant
            - Keep the response informative but concise
            """

            final_response = self.llm.generate_response(prompt)

        else:
            # Direct response from LLM
            final_response = action_results["results"]

        return final_response

    def process_query(self, query: str) -> str:
        """
        Main method that implements the complete ReAct cycle.

        Args:
            query (str): User's input query

        Returns:
            str: Final response
        """
        print("="*60)
        print(" Starting ReAct Process")
        print("="*60)

        try:
            # Step 1: REASON
            reasoning = self.reason(query)

            # Step 2: ACT
            action_results = self.act(reasoning)

            # Step 3: OBSERVE and respond
            final_response = self.observe_and_respond(query, action_results)

            print(f"\n FINAL RESPONSE:")
            print("-" * 40)
            return final_response

        except Exception as e:
            error_msg = f" Error in ReAct process: {str(e)}"
            print(error_msg)
            return error_msg

print(" Pure ReAct Agent class loaded with Serper Tool!")

 Pure ReAct Agent class loaded with Serper Tool!


## Initialize and Test the Agent

Let's create our ReAct agent and test it with some example queries.

In [26]:
# Initialize the ReAct agent
agent = PureReActAgentSerper()

ReAct Agent initialized successfully!
Ready to reason and act on your queries!


In [34]:
agent = PureReActAgentTavily()

ReAct Agent initialized successfully!
Ready to reason and act on your queries!


## Example Queries

Let's test our agent with different types of queries to see how it decides between web search and built-in knowledge.

In [12]:
# Example 1: Technical question
query1 = "Write a FastAPI API for inferencing an ML model saved via pickle or joblib."
response1 = agent.process_query(query1)
print(response1)

 Starting ReAct Process

 REASONING about: 'Write a FastAPI API for inferencing an ML model saved via pickle or joblib.'
Using built-in knowledge (no search needed)
💭 Decision: direct_response

⚡ ACTING: direct_response

 OBSERVING results and generating response...

 FINAL RESPONSE:
----------------------------------------
```python
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import pickle
import joblib
import pandas as pd
import io  # For handling file uploads
from typing import Union  # For supporting both pickle and joblib

app = FastAPI()

# Define a class to hold the model and its type
class ModelHolder:
    model = None
    model_type = None # "pickle" or "joblib"

model_holder = ModelHolder()


# Define a Pydantic model for input data
class InputData(BaseModel):
    data: dict  # Allow a dictionary for flexibility, adjust based on your model's expected input


# Endpoint to load the model
@app.post("/load-model/")
async def load_m

In [35]:
# Example 2: Current event question (should trigger web search)
query2 = "What's latest on Mohsin Naqvi?"
response2 = agent.process_query(query2)
print(response2)

 Starting ReAct Process

 REASONING about: 'What's latest on Mohsin Naqvi?'
Search triggered by temporal keyword (recent)
💭 Decision: web_search

⚡ ACTING: web_search

 OBSERVING results and generating response...

 FINAL RESPONSE:
----------------------------------------
The latest news surrounding Mohsin Naqvi, Chairman of the Pakistan Cricket Board (PCB) and ACC Chief, revolves around the Asia Cup and its aftermath.

*   **Asia Cup Controversy:** Mohsin Naqvi is facing criticism and controversy related to the Asia Cup, leading to calls for his resignation (India Today).
*   **Shahid Afridi's Call for Resignation:** Shahid Afridi has publicly urged Mohsin Naqvi to step down as PCB chairman following the Asia Cup debacle (India Today).
*   **Apology to BCCI:** Mohsin Naqvi has reportedly apologized to the Board of Control for Cricket in India (BCCI) in a meeting on Tuesday, September 30 (India TV News, News18, Hindustan Times).
*   **Trophy Handover:** The Asia Cup trophy has been han

In [16]:
# Example 3: Latest developments (should trigger web search)
query3 = "What are the latest developments in AI?"
response3 = agent.process_query(query3)
print(response3)

 Starting ReAct Process

 REASONING about: 'What are the latest developments in AI?'
Search triggered by temporal keyword (recent)
💭 Decision: web_search

⚡ ACTING: web_search

 OBSERVING results and generating response...

 FINAL RESPONSE:
----------------------------------------
Based on the provided search results, here's a summary of the latest developments in AI:

*   **Autonomous AI Agents:** AI is increasingly being used to create autonomous systems that can make decisions and perform tasks on behalf of users (Appinventiv).

*   **Improved AI Models:** The open-source AI model DeepCogito v2 has been released, showing improved logical reasoning and task planning capabilities compared to other closed source models (Crescendo).

*   **AI-Powered Video Generation:** OpenAI has launched a new social media app for creating short videos with audio from text, posing a potential challenge to platforms like TikTok and YouTube (WSJ). Google has also released its Veo 3 AI video creation too

## Summary

This Pure Python ReAct Agent demonstrates:

1. **Intelligent Decision Making**: Uses pattern recognition to decide when web search is needed
2. **ReAct Pattern**: Implements Reason → Act → Observe cycle
3. **Multi-Modal Responses**: Can both search the web and use built-in knowledge
4. **Temporal Awareness**: Recognizes time-sensitive queries
5. **Topic Classification**: Identifies domains that require real-time information

**Key Components:**
- `SearchTriggerIntelligence`: Decides when to search vs use knowledge
- `WebSearchTool`: Handles web search via Serper API or Tavily API Key
- `GeminiLLM`: Provides reasoning and response generation
- `PureReActAgent`: Orchestrates the complete ReAct workflow

This approach allows for a more intelligent and context-aware AI agent that can handle both factual questions and current events effectively.